In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
import preprocess
from scipy.stats import pearsonr, spearmanr, ks_2samp
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.feature_selection import chi2
from sklearn.model_selection import KFold

importing Jupyter notebook from preprocess.ipynb


In [3]:
#ks and chi-squared test
def test(mat):
     #KS test
    #null hypothesis: two variables are from the same distribution
    cat_cols = []
    num_cols = []
    for i in mat.columns:
        if mat[i].nunique() <= 2:
            cat_cols.append(i)
        else:
            num_cols.append(i)
    num_cols = set(num_cols)
    num_cols.add('ssi')

    df = mat[num_cols]
    ks = {}
    for i in df.columns:
        if i != "ssi":
            ks[i] = ks_2samp(df.loc[df.ssi==1, i], df.loc[df.ssi==0, i])[1]
            #ks.append((i, ks_2samp(df.loc[df.ssi==1, i], df.loc[df.ssi==0, i])[1]))
    ks_sel = ks.items()
    ks_sel = [i[0] for i in ks_sel if i[1] <= 0.02]  #selecting variables rejecting null hypothesis
    
    #chi squared test
    # null hypothesis: two variables are independent
    df = mat[cat_cols]
    chi_scores = chi2(df.drop(['ssi'], axis = 1),df['ssi'])

    chic = pd.Series(chi_scores[1], index = df.drop(['ssi'], axis = 1).columns).sort_values(axis = 0)
    chi_cols = chic[chic < 0.02].index   #selecting variables rejecting null hypothesis 

    cols = set(ks_sel) | set(chi_cols)
    cols.add("ssi")
    
    return cols

#ks test    
def test2(df):
    ks = {}
    for i in df.columns:
        if i != "ssi":
            ks[i] = ks_2samp(df.loc[df.ssi==1, i], df.loc[df.ssi==0, i])[1]
            #ks.append((i, ks_2samp(df.loc[df.ssi==1, i], df.loc[df.ssi==0, i])[1]))
    ks_sel = ks.items()
    ks_sel = [i[0] for i in ks_sel if i[1] <= 0.05]  #selecting variables rejecting null hypothesis
    cols = set(ks_sel)
    cols.add("ssi")
    return cols

#checking correlation
def get_cor_feat(cor):    
    feats = cor.index
    cor_thresh = 0.3
    flag = [True]*len(cor.index)

    for i in range(len(cor.index)):
        if flag[i] != True:
            continue
        for j in range(i+1, len(cor.index)):
            if flag[i] != True:
                break
            if flag[j] != True:
                continue
            if feats[i] != "ssi" and feats[j] != "ssi" and i != j:
                #if the correlation b/w 2 features is greater than threshold, reject the one 
                #that is less correlated with the target
                if abs(cor.iloc[i, j]) > cor_thresh:
                    if abs(cor.loc[feats[i], "ssi"]) > abs(cor.loc[feats[j], "ssi"]):
                        flag[j] = False
                    else:
                        flag[i] = False
    cor2 = cor.columns[flag]
    return cor2

In [6]:
def get_fin_cols(cov):
    #cols = test(cov)
    cols = test2(cov)

    cor = cov[cols].corr()

    cor_feats = get_cor_feat(cor)

    cor.reset_index(inplace = True)
#     cor.drop(['level_0'], axis = 1, inplace = True)

    #VIF
    cov = cov[cor_feats]    
    df = sm.add_constant(cov)
    #rm = ['vitals_pulse_diff', 'vitals_temp_diff', 'index']
    rm = []
    df.drop(rm, axis = 1, inplace = True)
    vif_df = [[df.columns[i], variance_inflation_factor(df.values, i)] for i in range(df.shape[1])]

    '''vifs = []
    ssi_index = list(cor.columns).index("ssi")
    for i in vif_df:
        if i[1] > 5 and i[0] != 'const':
            vifs.append(i + [abs(cor.loc[ssi_index, i[0]])])

    print(pd.DataFrame(vifs, columns = ["col", "vif", "cor"]))'''

    fin_cols = set()
    for c in vif_df:
        if c[0] != "const" and c[0] != "index":
            fin_cols.add(c[0])

    return fin_cols

In [7]:
cov_train, cov_test, cov_val = #load from file

cov_train_final, cov_test_final, cov_val_final = preprocess.preprocess(cov_train, cov_test, cov_val)

fin_cols = get_fin_cols(cov_train_final)
cov_train_final[fin_cols].to_csv("selected_columnsks_ks_0.02_chi_0.02_cor_0.3.csv")

In [ ]:
len(fin_cols)